Prepare Data

In [ ]:
# Clone repo
!git clone https://github.com/benadar293/benadar293.github.io.git

# Install requirements
%cd repository
!pip install -r requirements.txt

Cloning into 'benadar293.github.io'...
remote: Enumerating objects: 977, done.
remote: Counting objects: 100% (976/976), done.
remote: Compressing objects: 100% (501/501), done.
remote: Total 977 (delta 474), reused 936 (delta 457), pack-reused 1 (from 1)
Receiving objects: 100% (977/977), 418.27 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (474/474), done.
Updating files: 100% (75/75), done.


In [ ]:
# Create pitch shifted copies of the performance, from -5 to 5 semitones (11 copies).

import os
import librosa
import soundfile as sf
from glob import glob
import numpy as np


# src_dir = '/path/to/performance'
src_dir = 'MusicNetSamples'
target_root = 'NoteEM_audio'
# file_type = '.mp3'
file_type = '.flac'
# file_type = '.wav'

audio_src_files = glob(src_dir + '/**/*' + file_type, recursive=True)
audio_src_files = sorted(audio_src_files)

print('Beginning pitch shift from', src_dir)
for f in audio_src_files:
    print(f)
    f_split = f.split('/')
    piece, part = f_split[-2:]
    try:
        assert '/'.join(f_split[: -1]) == src_dir
    except AssertionError as e:
        print('/'.join(f_split[: -1]))
        print(src_dir)
        raise e
    for shift in range(-5, 6):
        print(shift)
        os.makedirs(target_root + '/' + piece + '#' + str(shift), exist_ok=True)
        suffix = part[-len(file_type):]
        assert suffix == file_type
        f_target1 = target_root + '/' + piece + '#' + str(shift) + '/' + part.replace(file_type, '#{}.flac'.format(shift))
        f_target2 = target_root + '/' + piece + '#' + str(shift) + '/' + part[: -len(file_type)] + '#{}.flac'.format(shift)
        assert f_target1 == f_target2
        f_target = f_target1
        command = 'sox \"' + f + '\" -r 16000 \"' + f_target + '\" pitch {}'.format(100 * shift)

        # if you want to add a small shift (<= 0.1 semitone) use this command instead:
        # small_shift = np.random.randint(-10, 11)
        # command = 'sox \"' + f + '\" -r 16000 \"' + f_target + '\" pitch {}'.format(100 * shift + small_shift)

        print('command:', command)
        os.system(command)

In [ ]:
# Convert midi to a note-list representation in a tsv file. Each line in the tsv file will contain
# information of a single note: onset time, offset time, note number, velocity, and instrument.

import numpy as np
import os
import warnings
from onsets_and_frames.midi_utils import parse_midi_multi
warnings.filterwarnings("ignore")


def midi2tsv_process(midi_path, target_path, shift=0, force_instrument=None):
    midi = parse_midi_multi(midi_path, force_instrument=force_instrument)
    print(target_path)
    if shift != 0:
        midi[:, 2] += shift
    np.savetxt(target_path, midi,
               fmt='%1.6f', delimiter='\t', header='onset,offset,note,velocity,instrument')


# midi_src_pth = '/path/to/midi/perfromance'
midi_src_pth = 'MusicNetSamples'

target = '/disk4/ben/UnalignedSupervision/NoteEM_tsv'


FORCE_INSTRUMENT = None
piece = midi_src_pth.split('/')[-1]
os.makedirs(target + '/' + piece, exist_ok=True)
for f in os.listdir(midi_src_pth):
    if f.endswith('.mid') or f.endswith('.MID'):
        print(f)
        midi2tsv_process(midi_src_pth + '/' + f,
                         target + '/' + piece + '/' + f.replace('.mid', '.tsv').replace('.MID', '.tsv'),
                         force_instrument=FORCE_INSTRUMENT)

Train

In [ ]:
!python train.py